By now, I have extracted a multitude of features and applied Gaussian Mixture Unsupvised modelling to all of them I have noted some trends that hint at the presence of cohesive and non-cohesive groups in the dataset. To better intepret the networks of interaction constituting meetings, I will now shift my attention to Graph Neural Networks. These architectures learn a graphs inherent struture using node and edge features and can then predict the state of nodes, edges, and complete graphs. Once I have built GNN's I will move on to spatio-temporal GNNs and inetepret the progression of teams. Spatio-temporal GNNs add a temporal aspect to GNNs through transformers, LSTMs or other sequence based architectures. 

To start doing all of this, I will first build graphs of each meeting in a flexible way that will allow me to adjust the time segments graphs are based off. 

Graphs will have node features and edge features where each node will be a participant and each edge will decribe the relatioship between two participants in conversation. 

Node Features (extracted for each paricipant): 
1. Portion of Time spoken
2. number of speaking turns longer than S seconds divided by segment length. 
3. rate of speech (WPS)
4. Lexical Richness: Ratio of unique words to total words spoken.
5. Positive Sentiment Proportion: Portion of words with positive sentiment.
6. Negative Sentiment Proportion: Portion of words with negative sentiment.
7. Complexity of Language: Proportion of complex words (words with more than three syllables) used.
8. First Person Pronoun Usage: Proportion of first-person pronouns (I, me, my, mine) to total words spoken.
9. Second Person Pronoun Usage: Proportion of second-person pronouns (you, your, yours) to total words spoken.
10. Third Person Pronoun Usage: Proportion of third-person pronouns (he, she, they, etc.) to total words spoken.
11. Agreement Words Proportion: Portion of words indicating agreement (yes, definitely, absolutely, etc.).
12. Disagreement Words Proportion: Portion of words indicating disagreement (no, never, disagree, etc.).

Edge Features will be directed: From partucivant U towards participant V
1. LSM Score between U and V (Mutual feature)
2. Number of times U interrupts V
3. Average amount of time between V's turn and U's subsequent turn. 

In [12]:
import pandas as pd 
import os

First, we combine all datatframes containing meeting dialogue into one massive data frame and create a function that iterates through each meeting and subsequently through sgment chunks in each meeting of a user specified length. 

In [13]:
base_dir = '../words'  

dialogues = pd.DataFrame()

# Initialize group index
group_index = 0

# Iterate through each folder in the base directory
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    # only for meeting scenarios
    if folder_name[1] == 'S':
        csv_file = f"{folder_name}.csv"
        file_path = os.path.join(folder_path, csv_file)
        df = pd.read_csv(file_path)
        dialogues = pd.concat([dialogues, df], ignore_index=True)
        
dialogues.head()

,Start,End,Speaker,Speech,meeting
0,13.95,14.16,B,Hi .,TS3007a
1,14.71,15.20,C,Hi .,TS3007a
2,15.74,16.25,A,Hello .,TS3007a
3,17.61,17.79,B,Oh .,TS3007a
4,41.74,42.36,A,Good morning .,TS3007a


In [14]:
dialogues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162976 entries, 0 to 162975
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Start    162976 non-null  float64
 1   End      162976 non-null  float64
 2   Speaker  162976 non-null  object 
 3   Speech   162976 non-null  object 
 4   meeting  162976 non-null  object 
dtypes: float64(2), object(3)
memory usage: 6.2+ MB


In [15]:
def extract_lexical_richness(node_dic, df):
    for speaker in node_dic.keys():
        speaker_df = df[df['Speaker'] == speaker]
        total_words = sum(speaker_df['Speech'].apply(lambda x: len(x.split())))
        unique_words = len(set(" ".join(speaker_df['Speech']).split()))
        lexical_richness = unique_words / total_words if total_words > 0 else 0
        node_dic[speaker].append(('lexical_richness', lexical_richness))

In [16]:
from textblob import TextBlob
def extract_sentiment_proportion(node_dic, df, sentiment_type, threshold):
    for speaker in node_dic.keys():
        speaker_df = df[df['Speaker'] == speaker]
        total_words = 0
        sentiment_count = 0
        for speech in speaker_df['Speech']:
            blob = TextBlob(speech)
            for word in blob.words:
                word_sentiment = TextBlob(word).sentiment.polarity
                total_words += 1
                if sentiment_type == "positive" and word_sentiment > threshold:
                    sentiment_count += 1
                elif sentiment_type == "negative" and word_sentiment < threshold:
                    sentiment_count += 1
        sentiment_proportion = sentiment_count / total_words if total_words > 0 else 0
        node_dic[speaker].append((f'{sentiment_type}_sentiment_proportion', sentiment_proportion))

In [17]:
def extract_disagreement_words_proportion(node_dic, df):
    disagreement_words = {
                                    'no', 'not', 'never', 'don\'t', 'do not', 'can\'t', 'cannot', 'won\'t', 'will not', 'isn\'t', 'is not', 'aren\'t', 'are not', 
                                    'couldn\'t', 'could not', 'shouldn\'t', 'should not', 'wouldn\'t', 'would not', 'doesn\'t', 'does not'
                                }
    
    for speaker in node_dic.keys():
        speaker_df = df[df['Speaker'] == speaker]
        total_words = sum(speaker_df['Speech'].apply(lambda x: len(x.split())))
        disagreement_count = sum(speaker_df['Speech'].apply(lambda x: len([word for word in x.split() if word.lower() in disagreement_words])))
        disagreement_words_proportion = disagreement_count / total_words if total_words > 0 else 0
        node_dic[speaker].append(('disagreement_words_proportion', disagreement_words_proportion))

In [18]:
def extract_agreement_words_proportion(node_dic, df):
    agreement_words_phrases = {
        'yes', 'agree', 'do', 'can', 'will', 'is', 'are', 'could', 'should', 'would', 'does'
    }

    for speaker in node_dic.keys():
        speaker_df = df[df['Speaker'] == speaker]
        total_words = sum(speaker_df['Speech'].apply(lambda x: len(x.split())))
        agreement_count = sum(speaker_df['Speech'].apply(lambda x: len([word for word in x.split() if word.lower() in agreement_words_phrases])))
        agreement_words_proportion = agreement_count / total_words if total_words > 0 else 0
        node_dic[speaker].append(('agreement_words_proportion', agreement_words_proportion))

In [19]:
from wordfreq import word_frequency
from nltk.corpus import stopwords
import nltk

# Download the stopwords from NLTK if not already downloaded
nltk.download('stopwords')

def extract_complexity_of_language(node_dic, df):
    stop_words = set(stopwords.words('english'))

    def word_rarity(word):
        # Return the frequency of the word
        return word_frequency(word, 'en')

    for speaker in node_dic.keys():
        speaker_df = df[df['Speaker'] == speaker]
        total_rarity = 0
        valid_word_count = 0

        for speech in speaker_df['Speech']:
            words = speech.split()
            for word in words:
                word = word.lower()
                if word not in stop_words:
                    rarity = word_rarity(word)
                    total_rarity += rarity
                    valid_word_count += 1

        average_rarity = total_rarity / valid_word_count if valid_word_count > 0 else 0
        node_dic[speaker].append(('average_word_rarity', average_rarity))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manuelnunezmartinez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
def extract_first_person_pronoun_usage(node_dic, df):
    first_person_pronouns = {'I', 'me', 'my', 'mine'}
    for speaker in node_dic.keys():
        speaker_df = df[df['Speaker'] == speaker]
        total_words = sum(speaker_df['Speech'].apply(lambda x: len(x.split())))
        first_person_count = sum(speaker_df['Speech'].apply(lambda x: len([word for word in x.split() if word.lower() in first_person_pronouns])))
        first_person_pronoun_usage = first_person_count / total_words if total_words > 0 else 0
        node_dic[speaker].append(('first_person_pronoun_usage', first_person_pronoun_usage))

def extract_second_person_pronoun_usage(node_dic, df):
    second_person_pronouns = {'you', 'your', 'yours'}
    for speaker in node_dic.keys():
        speaker_df = df[df['Speaker'] == speaker]
        total_words = sum(speaker_df['Speech'].apply(lambda x: len(x.split())))
        second_person_count = sum(speaker_df['Speech'].apply(lambda x: len([word for word in x.split() if word.lower() in second_person_pronouns])))
        second_person_pronoun_usage = second_person_count / total_words if total_words > 0 else 0
        node_dic[speaker].append(('second_person_pronoun_usage', second_person_pronoun_usage))

def extract_third_person_pronoun_usage(node_dic, df):
    third_person_pronouns = {'he', 'she', 'they', 'him', 'her', 'them', 'his', 'hers', 'theirs'}
    for speaker in node_dic.keys():
        speaker_df = df[df['Speaker'] == speaker]
        total_words = sum(speaker_df['Speech'].apply(lambda x: len(x.split())))
        third_person_count = sum(speaker_df['Speech'].apply(lambda x: len([word for word in x.split() if word.lower() in third_person_pronouns])))
        third_person_pronoun_usage = third_person_count / total_words if total_words > 0 else 0
        node_dic[speaker].append(('third_person_pronoun_usage', third_person_pronoun_usage))

In [44]:
def extract_time_portion(node_dic, df):
    total_time = df['End'].max() - df['Start'].min()

    for speaker in node_dic.keys():
        speaker_df = df[df['Speaker'] == speaker]
        total_speaking_time = (speaker_df['End'] - speaker_df['Start']).sum()
        time_portion = total_speaking_time / total_time if total_time > 0 else 0
        node_dic[speaker].append(('time_portion', time_portion))

In [22]:
def extract_rate_of_speech(node_dic, df):
    for speaker in node_dic.keys():
        speaker_df = df[df['Speaker'] == speaker]
        total_words = sum(speaker_df['Speech'].apply(lambda x: len(x.split())))
        total_speaking_time = (speaker_df['End'] - speaker_df['Start']).sum()
        rate_of_speech = total_words / total_speaking_time if total_speaking_time > 0 else 0
        node_dic[speaker].append(('rate_of_speech', rate_of_speech))

In [43]:
def extract_speaking_turns(node_dic, df, min_duration=1):
    # Filter the DataFrame to only include rows where the speaking duration is at least `min_duration` seconds
    filtered_df = df[(df['End'] - df['Start']) >= min_duration]
    total_turns = len(filtered_df)

    for speaker in node_dic.keys():
        speaker_df = filtered_df[filtered_df['Speaker'] == speaker]
        speaking_turns = len(speaker_df)
        turns_proportion = speaking_turns / total_turns if total_turns > 0 else 0
        node_dic[speaker].append(('speaking_turns_proportion', turns_proportion))

In [24]:
import heapq


def clean_and_merge_intervals(data):
    cleaned_data = [([*set(speakers)], time_range) for speakers, time_range in data]

    merged_data = []
    for i, (speakers, (start, end)) in enumerate(cleaned_data):
        if merged_data:
            last_speakers, (last_start, last_end) = merged_data[-1]
            if last_speakers == speakers and last_end == start:
                # Merge current with the last one
                merged_data[-1] = (last_speakers, (last_start, end))
            else:
                merged_data.append((speakers, (start, end)))
        else:
            merged_data.append((speakers, (start, end)))

    return merged_data


def get_overlaps(df):

    starts = df['Start'].tolist()
    ends = df['End'].tolist()
    speakers = df['Speaker'].tolist()

    turns = [((start, end), speaker) for start, end, speaker in zip(starts, ends, speakers)]


    overlaps = []
    heap = []
    heapq.heapify(heap)

    for i, ((start, end), speaker) in enumerate(turns):

        heapq.heappush(heap, (end,(start,speaker)))

        next_start = 0
        if i < len(turns) - 1:
            next_start = turns[i+1][0][0]
        
            
            substart = start
            while len(heap) > 0 and heap[0][0] < next_start:
                min_elem = heap[0][0]
                overlaps.append(([label for (_ , (_ ,  label)) in list(heap)], (substart, min_elem)))
                substart = min_elem
                heapq.heappop(heap)
            if len(heap) > 0:
                overlaps.append(([label for (_ , (_ ,  label)) in list(heap)], (start, min(next_start, end))))
        else:
            substart = start
            while len(heap) > 0:
                min_elem = heap[0][0]
                overlaps.append(([label for (_ , (_ ,  label)) in list(heap)], (substart, min_elem)))
                substart = min_elem
                heapq.heappop(heap)

    overlaps = clean_and_merge_intervals(overlaps)

    return overlaps


In [40]:
def extract_interruptions(edge_dic, df):
    # Initialize the adjacency matrix for interruptions
    interruptions_matrix = {
        'A,B': 0, 'A,C': 0, 'A,D': 0,
        'B,A': 0, 'B,C': 0, 'B,D': 0,
        'C,A': 0, 'C,B': 0, 'C,D': 0,
        'D,A': 0, 'D,B': 0, 'D,C': 0
    }

    speaker_intervals = get_overlaps(df)
    num_interruptions = 0

    for i in range(len(speaker_intervals) - 1):
        current_interval = speaker_intervals[i]
        next_interval = speaker_intervals[i + 1]
        
        current_speakers = current_interval[0]
        next_speakers = next_interval[0]
        current_end = current_interval[1][1]
        next_start = next_interval[1][0]
        
        # Check if there is no gap between intervals
        if current_end == next_start:
            # Check if there is only one speaker in the current interval
            if len(current_speakers) == 1:
                original_speaker = current_speakers[0]
                # Check if the next interval has multiple speakers including the original one
                if original_speaker in next_speakers and len(next_speakers) > 1:
                    for speaker in next_speakers:
                        if speaker != original_speaker:
                            # Update the interruptions count
                            num_interruptions += 1
                            interruptions_matrix[f'{speaker},{original_speaker}'] += 1

    for key in edge_dic.keys():
        if key in interruptions_matrix:
            if num_interruptions > 0:
                edge_dic[key].append(('interruptions', interruptions_matrix[key]/num_interruptions))
            else:
                edge_dic[key].append(('interruptions', interruptions_matrix[key]))

In [26]:
import spacy
nlp = spacy.load('en_core_web_sm')
from itertools import combinations


# Define function words lists
adverbs = ["here", "there", "where", "when", "why", "how", "up", "down", "still", "too", "very"]
articles = ["a", "an", "the"]
auxiliary_verbs = ["be", "is", "are", "was", "were", "been", "being", "do", "does", "did", "have", "has", "had", "having", "will", "would", "shall", "should", "can", "could", "may", "might", "must"]
conjunctions = ["and", "but", "or", "nor", "for", "yet", "so"]
indefinite_pronouns = ["all", "another", "any", "anybody", "anyone", "anything", "each", "either", "everybody", "everyone", "everything", "few", "many", "more", "most", "much", "neither", "nobody", "none", "no one", "nothing", "one", "other", "others", "several", "some", "somebody", "someone", "something"]
personal_pronouns = ["I", "me", "we", "us", "you", "he", "him", "she", "her", "it", "they", "them"]
negations = ["not", "no", "never", "none"]
prepositions = ["of", "in", "to", "for", "with", "on", "at", "by", "from", "about", "as", "into", "like", "through", "after", "over", "between", "out", "against", "during", "without", "before", "under", "around", "among"]
quantifiers = ["some", "any", "all", "both", "each", "every", "few", "many", "more", "most", "much", "several", "enough", "plenty", "a lot of", "lots of"]

function_word_types = [
    'Adverbs', 'Articles', 'Auxiliary Verbs', 'Conjunctions', 
    'Indefinite Pronouns', 'Personal Pronouns', 'Negations', 
    'Prepositions', 'Quantifiers'
]
    

# Define a function to count function words
def count_function_words(doc, word_list):
    return sum([1 for token in doc if token.text.lower() in word_list])

# Define a function to count words
def count_words(speech):
    if pd.isna(speech):
        return 0
    return len(speech.split())

In [27]:
def get_LIWC(speaker1_count, speaker2_count):
    lsm = 1 - abs(speaker1_count - speaker2_count) / (speaker1_count + speaker2_count + 0.0001)
    return lsm

def compute_lsm_cl(df, speaker1, speaker2):
    lsm_values = []

    for word_type in function_word_types:
        speaker1_count = df[df['Speaker'] == speaker1][word_type].sum()
        speaker2_count = df[df['Speaker'] == speaker2][word_type].sum()
        
        lsm_values.append(get_LIWC(speaker1_count, speaker2_count))
    
    average_lsm = sum(lsm_values) / len(lsm_values)
    
    return average_lsm


def extract_lsm_score(edge_dic, df):
    LSM_df = df.copy()

    for category in function_word_types:
        LSM_df[category] = 0


    for index, row in LSM_df.iterrows():
        speech = row['Speech']
        if pd.isna(speech):
            continue
        doc = nlp(speech)
        LSM_df.at[index, 'Adverbs'] = count_function_words(doc, adverbs)
        LSM_df.at[index, 'Articles'] = count_function_words(doc, articles)
        LSM_df.at[index, 'Auxiliary Verbs'] = count_function_words(doc, auxiliary_verbs)
        LSM_df.at[index, 'Conjunctions'] = count_function_words(doc, conjunctions)
        LSM_df.at[index, 'Indefinite Pronouns'] = count_function_words(doc, indefinite_pronouns)
        LSM_df.at[index, 'Personal Pronouns'] = count_function_words(doc, personal_pronouns)
        LSM_df.at[index, 'Negations'] = count_function_words(doc, negations)
        LSM_df.at[index, 'Prepositions'] = count_function_words(doc, prepositions)
        LSM_df.at[index, 'Quantifiers'] = count_function_words(doc, quantifiers)

    speakers = ['A', 'B', 'C', 'D']
    
    for speaker1 in speakers:
        for speaker2 in speakers:
            if speaker1 != speaker2:
                key = f'{speaker1},{speaker2}'
                lsm_value = compute_lsm_cl(LSM_df, speaker1, speaker2)
                edge_dic[key].append(('LSM', lsm_value))

In [8]:
# def extract_average_time_between_turns(edge_dic, df, min_speaking_duration=3.0):
#     # Filter the dataframe to only include rows where the speaker speaks for at least min_speaking_duration seconds
#     df['Duration'] = df['End'] - df['Start']
#     df_filtered = df[df['Duration'] >= min_speaking_duration]

#     speakers = ['A', 'B', 'C', 'D']

#     for speaker1 in speakers:
#         for speaker2 in speakers:
#             if speaker1 != speaker2:
#                 key = f'{speaker1},{speaker2}'
#                 turns_between = []
#                 for i in range(len(df_filtered) - 1):
#                     if df_filtered.iloc[i]['Speaker'] == speaker2:
#                         next_turns = df_filtered.iloc[i + 1:].reset_index()
#                         next_speaker_index = next_turns[next_turns['Speaker'] == speaker1].index
#                         if len(next_speaker_index) > 0:
#                             turns_between.append(next_speaker_index[0] + 1)  # +1 to account for zero-based indexing
                
#                 if turns_between:
#                     average_turns = sum(turns_between) / len(turns_between)
#                 else:
#                     average_turns = -1  # Or set to 0 or some other default value

#                 edge_dic[key].append(('average_turns_between', average_turns))

In [28]:
from collections import defaultdict

def extract_average_time_between_turns(edge_dic, df):
    # Filter out overlaps
    intervals = get_overlaps(df)
    filtered_intervals = [interval for interval in intervals if (len(interval[0])==1)]

    # Initialize dictionary to store turn probabilities
    turn_counts = defaultdict(int)
    speakers = ['A', 'B', 'C', 'D']

    # Iterate through each speaker to find speaking turns
    for i, (current_speakers, _) in enumerate(filtered_intervals):
        
        current_speaker = current_speakers[0]  
        # Check if this speaker talks after another speaker
        if i > 0:
            previous_speakers, _ = filtered_intervals[i - 1]
            previous_speaker = previous_speakers[0]
            turn_counts[(current_speaker, previous_speaker)] += 1

    for speaker1 in speakers:
        for speaker2 in speakers:
            if speaker1 != speaker2:
                probability = (turn_counts[(speaker1, speaker2)] / len(filtered_intervals)) if len(filtered_intervals) > 0 else 0
                edge_dic[f'{speaker1},{speaker2}'].append(('average_turns_between', probability))


In [45]:
def create_graph(df):
    node_dic = {'A': [], 'B': [], 'C': [], 'D': []}
    edge_dic = {'A,B': [], 'A,C': [], 'A,D': [], 'B,A': [], 'B,C': [], 'B,D': [], 'C,A': [], 'C,B': [], 'C,D': [], 'D,A': [], 'D,B': [], 'D,C': []}

    # extract all individual features 
    extract_time_portion(node_dic, df)
    extract_speaking_turns(node_dic, df)
    extract_rate_of_speech(node_dic, df)

    # extract uniqueness of speech 
    extract_lexical_richness(node_dic, df)

    # find exttent of both positive and negative sentiment 
    extract_sentiment_proportion(node_dic, df, 'positive', 0.2)
    extract_sentiment_proportion(node_dic, df, 'negative', -0.2)
    
    # look at rarity of non-function words used 
    extract_complexity_of_language(node_dic, df)

    # references to self, others, etc. 
    extract_first_person_pronoun_usage(node_dic, df)
    extract_second_person_pronoun_usage(node_dic, df)
    extract_third_person_pronoun_usage(node_dic, df)

    #find extent of both agreement and disagreement 
    extract_agreement_words_proportion(node_dic, df)
    extract_disagreement_words_proportion(node_dic, df)

    # extract all edge features
    extract_lsm_score(edge_dic, df)  # LSM Score between U and V (Mutual feature)
    extract_interruptions(edge_dic, df)  # Number of times U interrupts V
    extract_average_time_between_turns(edge_dic, df)

    return node_dic, edge_dic


In [30]:
# Define the necessary functions
def create_graphs(df, meeting, time_length):
    # hold graph relevant to this meeting
    graphs = []

    start_time = df['Start'].min()
    end_time = df['Start'].max()
    
    results = []
    current_start = start_time
    
    while current_start <= end_time:
        current_end = current_start + time_length
        segment = df[(df['Start'] >= current_start) & (df['Start'] < current_end)]
        
        # Check if segment contains 4 unique speakers and each speaks for at least 10% of the time
        unique_speakers = segment['Speaker'].unique()
        if len(unique_speakers) == 4:
            total_time = segment['End'].sum() - segment['Start'].sum()
            speaker_times = segment.groupby('Speaker').apply(lambda x: x['End'].sum() - x['Start'].sum())
            if all(speaker_time / total_time >= 0.1 for speaker_time in speaker_times):
                node_dic, edge_dic = create_graph(segment)
                graphs.append([meeting, current_start, (node_dic, edge_dic)])
        
        current_start = current_end

    return graphs

In [47]:
# we will populate this list with graphs entries will be of type [meeeting name, time interval start, ({dictionary of node features}, {dictionary of edge features}) ]
graphs = []
# time length currently 4 minutes 
time_length = 240 
# partition all dialogues into meetings to start 
meeting_groups = dialogues.groupby('meeting')

for meeting, group in meeting_groups:
    graph_adds = create_graphs(group, meeting, time_length )
    graphs += graph_adds

/var/folders/r_/wwpxjnln7vx4bw2q6dxn_svm0000gn/T/ipykernel_56295/2505756107.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  speaker_times = segment.groupby('Speaker').apply(lambda x: x['End'].sum() - x['Start'].sum())
/var/folders/r_/wwpxjnln7vx4bw2q6dxn_svm0000gn/T/ipykernel_56295/2505756107.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  speaker_times = segment.groupby('Speaker').apply(lambda x:

In [50]:
for i in range(5):
    print(graphs[i])
    print()

['ES2002c', 1680.02, ({'A': [('time_portion', 0.27292836481197835), ('speaking_turns_proportion', 0.08333333333333333), ('rate_of_speech', 3.6003740648379132), ('lexical_richness', 0.49783549783549785), ('positive_sentiment_proportion', 0.008968609865470852), ('negative_sentiment_proportion', 0.004484304932735426), ('average_word_rarity', 0.0003129816814159291), ('first_person_pronoun_usage', 0.0), ('second_person_pronoun_usage', 0.021645021645021644), ('third_person_pronoun_usage', 0.0), ('agreement_words_proportion', 0.06060606060606061), ('disagreement_words_proportion', 0.012987012987012988)], 'B': [('time_portion', 0.5290539390845657), ('speaking_turns_proportion', 0.6666666666666666), ('rate_of_speech', 3.3207365120205927), ('lexical_richness', 0.4430992736077482), ('positive_sentiment_proportion', 0.04392764857881137), ('negative_sentiment_proportion', 0.01808785529715762), ('average_word_rarity', 0.00034122183771929823), ('first_person_pronoun_usage', 0.0), ('second_person_pron

In [51]:
import pickle

# Assuming 'graphs' is your list of graph data
with open('graphs_data.pkl', 'wb') as f:
    pickle.dump(graphs, f)

The graphs I contructed above were used for node level preidction in which we predicted whether english was the native language of speakers in a conversation. We will now build graphs that are meant for graph-level cohesion predictions. The process will be similar to the one above, but we must extract info for the time stamps that cohesion annotations are available for. 

In [33]:
df_cohesion = pd.read_pickle('../Cohesion_Annotations/Cohesion_split_data.pkl')
df_cohesion.head()

,Meeting,Start,End,Cohesion,Task_Cohesion,Social_Cohesion,Miscellaneous_Cohesion,Task_Kappa,Social_Kappa,Cohesion_Kappa,Task_Average,Social_Average,Cohesion_Average
0,IS1006d,25.0,27.0,"[[5.0, 4.0, 4.0, 3.0, 5.0, 6.0, 3.0, 4.0, 5.0,...","[[4.0, 4.0, 3.0, 3.0, 4.0, 5.0, 7.0], [1.0, 2....","[[5.0, 5.0, 6.0, 5.0, 7.0, 5.0, 6.0, 5.0, 5.0,...","[[2.0, 9.0, 6.0], [2.0, 9.0, 3.0], [2.0, 9.0, ...",0.033335,0.140622,0.298247,4.142857,5.836735,5.197368
1,IS1007b,1.0,3.0,"[[5.0, 2.0, -1.0, 2.0, 2.0, 3.0, 2.0, 1.0, 2.0...","[[2.0, -1.0, 2.0, 2.0, 1.0, 5.0, 7.0], [4.0, 4...","[[5.0, 2.0, 3.0, 2.0, 5.0, 2.0, 1.0, 9.0, 2.0,...","[[1.0, 1.0, 2.0], [1.0, 3.0, 2.0], [2.0, 9.0, ...",0.162190,0.355717,0.320634,3.450000,2.489796,2.675325
2,IS1003c,23.0,25.0,"[[6.0, 5.0, 5.0, 3.0, 3.0, 4.0, 2.0, 2.0, 4.0,...","[[5.0, 5.0, 3.0, 2.0, 2.0, 5.0, 6.0], [2.0, 3....","[[6.0, 3.0, 4.0, 4.0, 3.0, 5.0, 3.0, 3.0, 4.0,...","[[2.0, 9.0, 3.0], [2.0, 9.0, 2.0], [1.0, 9.0, ...",0.107829,-0.036448,0.082340,3.666667,3.627451,3.538462
3,IS1005c,8.0,10.0,"[[3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 3.0, 4.0,...","[[3.0, 3.0, 4.0, 3.0, 3.0, 1.0, 1.0], [2.0, -1...","[[3.0, 4.0, 4.0, 4.0, 3.0, 3.0, 1.0, 5.0, 2.0,...","[[1.0, 1.0, 4.0], [1.0, 9.0, 9.0], [1.0, 5.0, ...",0.129785,0.306105,0.375170,2.100000,3.276596,2.864865
4,IS1001a,7.0,9.0,"[[5.0, 2.0, 5.0, 2.0, 4.0, 4.0, 2.0, 2.0, 2.0,...","[[2.0, 5.0, 2.0, 2.0, 2.0, 3.0, 3.0], [4.0, 5....","[[5.0, 4.0, 4.0, 2.0, 6.0, 4.0, 2.0, 5.0, 3.0,...","[[1.0, 5.0, 4.0], [1.0, 4.0, 1.0], [1.0, 4.0, ...",0.479670,0.324020,0.481775,2.900000,3.740000,3.405063


In [35]:
sub_dfs = []
for index, row in df_cohesion.iterrows():
    # Extract the values from the current row in df_cohesion
    meeting_value = row['Meeting']
    start_time = row['Start'] * 60
    end_time = row['End'] * 60

    # Check if the 'Meeting' value is found in the 'meeting' column of the 'dialogues' DataFrame
    if meeting_value in dialogues['meeting'].values:
        # Get the sub-DataFrame where 'meeting' equals the current 'Meeting' and 'Start'/'End' times fall within range
        sub_df = dialogues[(dialogues['meeting'] == meeting_value) & 
                        (dialogues['Start'] >= start_time) & 
                        (dialogues['End'] <= end_time)]
        sub_dfs.append({'df': sub_df, 'meeting': meeting_value, 'start': start_time})
        

In [46]:
graphs_cohesion = []

for interval in sub_dfs:
    node_dic, edge_dic = create_graph(interval['df'])
    graphs_cohesion.append({'meeting': interval['meeting'], 
                            'start': interval['start'], 
                            'features': (node_dic, edge_dic)})

In [47]:
graphs_cohesion[0]

{'meeting': 'IS1006d',
 'start': 1500.0,
 'features': ({'A': [('time_portion', 0.4571015970098533),
    ('speaking_turns_proportion', 0.2926829268292683),
    ('rate_of_speech', 2.174317041442113),
    ('lexical_richness', 0.46153846153846156),
    ('positive_sentiment_proportion', 0.06818181818181818),
    ('negative_sentiment_proportion', 0.022727272727272728),
    ('average_word_rarity', 9.540409090909092e-05),
    ('first_person_pronoun_usage', 0.0),
    ('second_person_pronoun_usage', 0.017094017094017096),
    ('third_person_pronoun_usage', 0.0),
    ('agreement_words_proportion', 0.017094017094017096),
    ('disagreement_words_proportion', 0.017094017094017096)],
   'B': [('time_portion', 0.20489296636085916),
    ('speaking_turns_proportion', 0.0975609756097561),
    ('rate_of_speech', 2.9436152570480507),
    ('lexical_richness', 0.647887323943662),
    ('positive_sentiment_proportion', 0.03333333333333333),
    ('negative_sentiment_proportion', 0.0),
    ('average_word_rarity

In [48]:
import pickle

# Assuming 'graphs' is your list of graph data
with open('data/graphs_cohesion_data_corrected_test.pkl', 'wb') as f:
    pickle.dump(graphs_cohesion, f)